# 简介
在这个教程中，我们将介绍pyecharts包。  
  
Echarts是百度开发的一个javascript可视化脚本，他的特点在于可交互（就是你鼠标移动上去，图片可以作出相应的变化）。小旭学长在echarts gallery发布了多款图表，累计12万的浏览量，获得300+赞，可以点击[这个链接](https://gallery.echartsjs.com/explore.html?u=bd-167860219&type=work#sort=rank~timeframe=all~author=all)查看  
echarts属于门槛比较高的一种可视化方法，需要写javascript，而javascript又是基于html网页，另外数据的处理还要用python，等于说你要熟练使用得同时会js，html，python三种语言。现在pyecharts出现了，直接在python里就可以调用生成echarts图片，大部分的图表只需要用python就可以生成，非常方便！  
  
相比我们之前介绍的几种画图方法，各有各的优势，小旭学长的总结如下：  
>matplotlib：纯python出图，可批量出图，缺点是出的图片为静态图片无法交互  
folium：主要功能是绘制地图，javascript出图可交互，坐标系为wgs84，数据不需要转坐标  
echarts：可绘制各种图表，也能绘制地图，javascript出图可交互，但绘制地图时底图一般采用百度地图，需要转坐标系

那么，我们赶紧开始学习python的pyecharts包吧！

最近新冠肺炎的疫情是相当的不妙。  
小旭学长观察到，一些大型网站的疫情发布地图都是基于echarts的，那么我们基于pyecharts来实现一下数据地图可视化的操作吧  
提供的基础数据是：
<div class="alert alert-info"><h2>提供的基础数据是：</h2><p>   
    数据：<br>  
    不提供，我们数据从网上抓，无中生有<br>  

 </p></div>

# 数据获取

很多网站都提供了疫情分布情况，数据都是公开的我们直接抓就行。这里以腾讯新闻的疫情发布链接为例，观察网络链接可以找到数据获取的访问请求

<img src="resource/yiqing.png"  style="width:800px">  

好的我们用最简单的方式把它抓下来，我们做省份的可视化

In [1]:
import urllib
import json
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
datajson=json.loads(response.read().decode('utf8'))
datajson=json.loads(datajson['data'])
datajson

{'chinaTotal': {'confirm': 24377, 'suspect': 23260, 'dead': 492, 'heal': 920},
 'chinaAdd': {'confirm': 3906, 'suspect': 46, 'dead': 67, 'heal': 288},
 'lastUpdateTime': '2020-02-05 13:25:17',
 'areaTree': [{'name': '中国',
   'children': [{'name': '湖北',
     'children': [{'name': '武汉',
       'total': {'confirm': 8351, 'suspect': 0, 'dead': 362, 'heal': 368},
       'today': {'confirm': 1967, 'suspect': 0, 'dead': 49, 'heal': 65}},
      {'name': '黄冈',
       'total': {'confirm': 1645, 'suspect': 0, 'dead': 25, 'heal': 50},
       'today': {'confirm': 223, 'suspect': 0, 'dead': 6, 'heal': 11}},
      {'name': '孝感',
       'total': {'confirm': 1462, 'suspect': 0, 'dead': 18, 'heal': 6},
       'today': {'confirm': 342, 'suspect': 0, 'dead': 1, 'heal': 2}},
      {'name': '襄阳',
       'total': {'confirm': 735, 'suspect': 0, 'dead': 2, 'heal': 7},
       'today': {'confirm': 103, 'suspect': 0, 'dead': 1, 'heal': 0}},
      {'name': '荆州',
       'total': {'confirm': 713, 'suspect': 0, 'dead

In [2]:
#提取各省份的数据
import pandas as pd
provincedata = pd.DataFrame(datajson['areaTree'][0]['children'])
provincedata.head(5)

,name,children,total,today
0,湖北,"[{'name': '武汉', 'total': {'confirm': 8351, 'su...","{'confirm': 16678, 'suspect': 0, 'dead': 479, ...","{'confirm': 3156, 'suspect': 0, 'dead': 65, 'h..."
1,浙江,"[{'name': '温州', 'total': {'confirm': 364, 'sus...","{'confirm': 895, 'suspect': 0, 'dead': 0, 'hea...","{'confirm': 66, 'suspect': 0, 'dead': 0, 'heal..."
2,广东,"[{'name': '深圳', 'total': {'confirm': 289, 'sus...","{'confirm': 870, 'suspect': 0, 'dead': 0, 'hea...","{'confirm': 73, 'suspect': 0, 'dead': 0, 'heal..."
3,河南,"[{'name': '信阳', 'total': {'confirm': 138, 'sus...","{'confirm': 764, 'suspect': 0, 'dead': 2, 'hea...","{'confirm': 89, 'suspect': 0, 'dead': 0, 'heal..."
4,湖南,"[{'name': '长沙', 'total': {'confirm': 164, 'sus...","{'confirm': 661, 'suspect': 0, 'dead': 0, 'hea...","{'confirm': 68, 'suspect': 0, 'dead': 0, 'heal..."


In [3]:
#整理一下数据，把total里面的数据展开
data1 = pd.DataFrame(provincedata['total'].apply(lambda r:[r]).sum())
data1['name'] = provincedata['name']
data1.head(5)

,confirm,suspect,dead,heal,name
0,16678,0,479,520,湖北
1,895,0,0,72,浙江
2,870,0,0,32,广东
3,764,0,2,47,河南
4,661,0,0,37,湖南


好的，到这一步我们已经获取了数据

# 可视化

## 全国数据可视化

官方配置文档：[pyecharts的地理图表教程](https://pyecharts.org/#/zh-cn/geography_charts)  
首先我们要把数据整理成echarts认识的格式，就是如下：

In [4]:
data1[['name','confirm']].values

array([['湖北', 16678],
       ['浙江', 895],
       ['广东', 870],
       ['河南', 764],
       ['湖南', 661],
       ['江西', 548],
       ['安徽', 530],
       ['重庆', 366],
       ['江苏', 341],
       ['四川', 301],
       ['山东', 298],
       ['北京', 253],
       ['上海', 233],
       ['福建', 205],
       ['黑龙江', 190],
       ['陕西', 165],
       ['广西', 150],
       ['河北', 135],
       ['云南', 122],
       ['海南', 91],
       ['辽宁', 81],
       ['山西', 81],
       ['天津', 69],
       ['贵州', 64],
       ['甘肃', 57],
       ['吉林', 54],
       ['内蒙古', 42],
       ['宁夏', 34],
       ['新疆', 32],
       ['香港', 18],
       ['青海', 17],
       ['台湾', 11],
       ['澳门', 10],
       ['西藏', 1]], dtype=object)

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
#创建echarts对象c
c = (
    Map()#告诉echarts这个是Map形式的图表
    .add("确诊", data1[['name','confirm']].values, "china")#加一个数据，这个数据名叫”确诊“，数据的地图是echarts自带的china
    .set_global_opts(#对图表添加设置
        title_opts=opts.TitleOpts(title='疫情地图')#设置图表的名称
    )
)
#导出为html文件
c.render('疫情地图.html')

打开目录下的 [疫情地图.html](疫情地图.html) 文件，效果如下
<img src="resource/地图v1.png"  style="width:500px">  
但是，我们还要在加一些美化调整，一些参数可以看[echarts官方的配置项手册](https://www.echartsjs.com/zh/option.html#title)

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
c = (
    Map()
    .add("确诊", 
         data1[['name','confirm']].values, 
         "china",
         is_roam = False,#不可鼠标缩放和平移漫游
         zoom = 1.2,#当前视角的缩放比例
         is_map_symbol_show = False, # 是否显示标记图形
         label_opts = opts.LabelOpts(position = 'inside'),#标签尽量放在图形区域内
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title='疫情地图'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,#设定分段颜色显示
                                          pieces=[{'min': 10000,'label':'10000人以上'}, #设定分段的值
                                                  {'min': 1000, 'max': 9999,'label':'1000-9999人'},
                                                  {'min': 500, 'max': 999,'label':'500-999人'},
                                                  {'min': 100, 'max': 499,'label':'100-499人'},
                                                  {'min': 10, 'max': 99,'label':'10-99人'},
                                                  {'min': 1, 'max': 9,'label':'1-9人'}],
                                           range_color=["#b4e0f3","#70b4eb","#1482e5","#1c3fbf","#070093" ] #调整显示颜色
                                         ),
    )
)
c.render('疫情地图.html')


打开目录下的 [疫情地图.html](疫情地图.html) 文件，效果如下
<img src="resource/地图2.png"  style="width:500px">  

## 单个省份数据可视化

In [50]:
import pandas as pd
#提取省份的数据
province = '广东'
guangdongdata = pd.DataFrame(provincedata[provincedata['name'] == province]['children'].iloc[0])
#整理一下数据，把total里面的数据展开
data2 = pd.DataFrame(guangdongdata['total'].apply(lambda r:[r]).sum())
data2['name'] = guangdongdata['name']+'市'
data2.head(5)

,confirm,suspect,dead,heal,name
0,289,0,0,13,深圳市
1,237,0,0,3,广州市
2,69,0,0,1,珠海市
3,49,0,0,1,佛山市
4,44,0,0,0,东莞市


In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
c = (
    Map()
    .add("确诊", 
         data2[['name','confirm']].values, 
         province,
         is_roam = False,#不可鼠标缩放和平移漫游
         zoom = 1.2,#当前视角的缩放比例
         is_map_symbol_show = False, # 是否显示标记图形
         label_opts = opts.LabelOpts(position = 'inside'),#标签尽量放在图形区域内
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=province+'疫情地图'),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,#设定分段颜色显示
                                          pieces=[{'min': 200, 'label':'200人以上'},#设定分段的值
                                                  {'min': 100, 'max': 199,'label':'100-199人'},
                                                  {'min': 50, 'max': 99,'label':'50-99人'},
                                                  {'min': 10, 'max': 49,'label':'10-49人'},
                                                  {'min': 1, 'max': 9,'label':'1-9人'}],
                                           range_color=["#b4e0f3","#70b4eb","#1482e5","#1c3fbf","#070093" ] #调整显示颜色
                                         ),
    )
)
c.render(province+'疫情地图.html')


打开目录下的 [广东疫情地图.html](广东疫情地图.html) 文件，效果如下
<img src="resource/广东疫情地图.png"  style="width:500px">  